In [1]:
import time
import heapq
import numpy as np
from copy import deepcopy

In [2]:
class Opt_State:

    def __init__(self, start, goal, heuristic, pre_move = 'root'):
        self.state = np.array([start[:3], start[3:6], start[6:]])
        self.goal = np.array([goal[:3], goal[3:6], goal[6:]])
        self.pre_move = pre_move
        self.heuristic = heuristic
        self.g = 0
        if heuristic == 'misplace':
            self.f = self.g + self.NumMisplaced()
        elif heuristic == 'manhattan':
            self.f = self.g + self.ManhattanDist()
        self.FindZero()

    def __eq__(self, other):
        if isinstance(other, Opt_State):
            return np.array_equal(self.state, other.state)
        
    def __hash__(self):
        return hash(tuple(map(tuple, self.state)))

    def __lt__(self, other):
        return self.f < other.f

    def FindZero(self):
        index = np.where(self.state == 0)
        coor = list(zip(index[0], index[1]))
        return list(coor[0])

    def ValidMove(self, move):
        if move == 'up':
            if self.FindZero()[0] != 0:
                return True
        elif move == 'down':
            if self.FindZero()[0] != 2:
                return True
        elif move == 'left':
            if self.FindZero()[1] != 0:
                return True
        elif move == 'right':
            if self.FindZero()[1] != 2:
                return True
        else:
            print('invalid move')
            return False
    
    def SavedMove(self, move):
        if self.ValidMove(move):
            saved = deepcopy(self)
            row = self.FindZero()[0]
            col = self.FindZero()[1]
            if move == 'up':
                self.state[row][col] = self.state[row-1][col]
                self.state[row-1][col] = 0
                self.pre_move = 'up'
                self.g += 1
                if self.heuristic == 'misplace':
                    self.f = self.g + self.NumMisplaced()
                elif self.heuristic == 'manhattan':
                    self.f = self.g + self.ManhattanDist()
            elif move == 'down':
                self.state[row][col] = self.state[row+1][col]
                self.state[row+1][col] = 0
                self.pre_move = 'down'
                if self.heuristic == 'misplace':
                    self.f = self.g + self.NumMisplaced()
                elif self.heuristic == 'manhattan':
                    self.f = self.g + self.ManhattanDist()
            elif move == 'left':
                self.state[row][col] = self.state[row][col-1]
                self.state[row][col-1] = 0
                self.pre_move = 'left'
                if self.heuristic == 'misplace':
                    self.f = self.g + self.NumMisplaced()
                elif self.heuristic == 'manhattan':
                    self.f = self.g + self.ManhattanDist()
            elif move == 'right':
                self.state[row][col] = self.state[row][col+1]
                self.state[row][col+1] = 0
                self.pre_move = 'right'
                if self.heuristic == 'misplace':
                    self.f = self.g + self.NumMisplaced()
                elif self.heuristic == 'manhattan':
                    self.f = self.g + self.ManhattanDist()
            return saved
        else:
            return False
        
    def Expand(self):
        nodes = []
        action_list = ['up','down','left','right']
        loop_move = ['down','up','right','left']
        for action in range(0,4):
            if(self.ValidMove(action_list[action]) and self.pre_move != loop_move[action]):
                temp = deepcopy(self)
                temp.SavedMove(action_list[action])
                nodes.append(temp)
        return nodes

    def NumMisplaced(self):
      if self.state.shape != self.goal.shape:
        print("Dimension Error")
        return False
      else:
        num_rows, num_cols = self.state.shape
        count = 0
        for i in range(0,num_rows):
            for j in range(0,num_cols):
                if self.state[i][j] != self.goal[i][j] and self.goal[i][j] != 0:
                    count += 1
        return count
    
    def ManhattanDist(self):
      if self.state.shape != self.goal.shape:
        print("Dimension Error")
        return False
      else:
        manhattan_dist = 0
        num_rows, num_cols = self.state.shape
        for r in range(0, num_rows):
            for c in range(0, num_cols):
                if self.state[r][c] != 0:
                    goal_index = np.where(self.goal == self.state[r][c])
                    goal_coor = list(zip(goal_index[0], goal_index[1]))
                    goal_coor = list(goal_coor[0])
                    manhattan_dist += abs(r - goal_coor[0]) + abs(c - goal_coor[1])
        return manhattan_dist

In [3]:
def path_find(history, state):

    path = [state.pre_move]
    temp = state

    while temp.pre_move != 'root':
      temp = history[temp]
      path.append(temp.pre_move)
      
    return list(reversed(path))[1:]

def opt_ASearch(start):
    global n_expand
    queue = [start]
    Inqueue = {start:True}
    visited = {start:True}
    history = {start:'root'}

    while queue:
        heapq.heapify(queue)
        target = heapq.heappop(queue)
        if (target.state == target.goal).all():
            print('Solved')
            return path_find(history, target)
        Inqueue[target] = False
        for node in target.Expand():
            n_expand += 1
            if node not in visited:
                visited[node] = True
                history[node] = target
                if node not in Inqueue:
                    heapq.heappush(queue, node)
                    Inqueue[node] = True
    return False


In [4]:
def InputToArray(s):
    a = []
    for c in s:
        a.append(int(c))
    return a

def ValidInput(a):
    checker = [0,1,2,3,4,5,6,7,8]
    for i in a:
        if i in checker:
            checker.remove(i)
        else:
            return False
    if checker:
        return False
    return True

In [5]:
if __name__ == "__main__":

    history = []
    n_expand = 0

    MISPLACE = 'misplace'
    MANHATTAN = 'manhattan'
    algo = 'M'
    
    start_input = input('Please Enter the Start state of the puzzle (e.g. 724506831 for the start state): ')
    goal_input = input('Please Enter the goal state of the puzzle (e.g. 012345678 for the goal state): ')

    if start_input.isnumeric() and goal_input.isnumeric():
        start = InputToArray(start_input)
        goal = InputToArray(goal_input)
    else:
        print('Invalid Input for start state or goal state')
        print('Start state: ', start_input)
        print('Goal state: ', goal_input)
        raise
    
    if ValidInput(start) and ValidInput(goal):
        method = input('Please select a Heuristic \'N\' for Number of Misplaced, \'M\' for Manhattan Distance')
        if method == 'N':
            #Run time is about 0.9 seconds for misplace heuristic
            algo = MISPLACE
            start_state = Opt_State(start, goal, heuristic=MISPLACE)
            start_time = time.perf_counter()
            history = opt_ASearch(start_state)
            end_time = time.perf_counter()
        elif method == 'M':
            #Run time is about 0.5 seconds for manhattan heuristic
            algo = MANHATTAN
            start_state = Opt_State(start, goal, heuristic=MANHATTAN)
            start_time = time.perf_counter()
            history = opt_ASearch(start_state)
            end_time = time.perf_counter()
        print("Heuristic used: ", algo)
        print(f'Algorithm Run Time: {end_time - start_time} seconds')
        print("Path found:", history)
        print("Number of nodes expaned: ", n_expand)
        print('Number of moves:', len(history))
    else:
        print('Invalid Input for heuristic')
        print('Start state: ', start_input)
        print('Goal state: ', goal_input)
        raise

Solved
Heuristic used:  manhattan
Algorithm Run Time: 0.03665919997729361 seconds
Path found: ['down', 'right', 'up', 'up', 'left', 'left', 'down', 'right', 'up', 'left', 'down', 'right', 'down', 'left', 'up', 'up', 'right', 'down', 'right', 'down', 'left', 'left', 'up', 'right', 'right', 'down', 'left', 'up', 'left', 'down', 'right', 'right', 'up', 'up', 'left', 'left', 'down', 'right', 'up', 'right', 'down', 'left', 'left', 'up']
Number of nodes expaned:  291
Number of moves: 44
